In [1]:
import numpy as np
arr = np.array([91143957525, 72152561493, 16488575863, 78723844732,
39650992134, 49490708734, 95637955772, 91242543078,
68627322005, 60631370900, 69953849019, 131494761995,
62673807843, 59716054233, 69127367898, 39233755450,
125455457666, 109600488012, 7431205397, 4385706296,
75926220981, 97163031028, 72677334199, 36641911059,
14174545774, 47464857381, 101634801557, 17933990204,
98386511935, 74167797746, 82661475281, 99115221055,
133097039008, 5924378212, 124040343623, 134483600734,
31637525959, 134108093465, 98885399365, 113503340702,
127568717809, 22819095499, 89973086127, 52412306129,
68193868122, 105643387627, 131953791389, 83254262499,
71639839245, 104792498353, 111111451072, 77913256122,
87408310351, 48640876886, 117077153907, 5205956967,
27714472246, 34875750439, 100215038203, 82904267139,
64766351391, 116874708395, 24475216421, 40683855730,
45722868978, 98815406129, 91404099330, 97127512068,
23034155668, 96554614560, 83024886757, 62594243893,
23255895237, 109335666325, 84674225740, 13641808209,
21844939347, 30860664525, 78581286049, 13878598309,
51215938257, 120374029609, 10889332075, 15566462607,
29746343618, 55356465751, 51550673843, 113416258594,
1171023799, 136712225207, 59794904654, 34872918073,
98081973844, 75720369863, 80372265821, 91862303826,
113335018225, 75687253385, 11012189811, 84538732806], dtype = object)

Описание алгоритма:

Пусть на вход подана последовательность A = (a[0], a[1], ..., a[99]).
a[i + 1] = (a * a[i] + c) % m.
Отсюда a[i + 1] = a * a[i] + c (mod m) и
a[i + 2] - a[i + 1] = a * (a[i + 1] - a[i]) (mod m).
Поэтому найдём разности d[i] = a[i + 1] - a[i].

In [2]:
diff_arr = np.diff(arr)

Потом заметим, что d[i + 2] = a * d[i + 1] = a^2 * d[i] (mod m), откуда d[i + 2] * d[i] = a^2 * d[i]^2 = (a * d[i])^2 = d[i + 1]^2 (mod m). То есть d[i + 2] * d[i] - d[i + 1] * d[i + 1] кратно m - получили много кратных числа m.

In [3]:
mult_arr = abs(diff_arr[:-2] * diff_arr[2:] - diff_arr[1:-1] ** 2)

Осталось найти их НОД:

In [4]:
M = np.gcd.reduce(mult_arr)

Здесь нет точной гарантии, что НОД M таких разностей будет именно m (вдруг M будет всего лишь кратным числа m?). Посмотрим на M и максимальный элемент A.

In [5]:
print(M, max(arr[1:]), M / max(arr[1:]))

137438953447 136712225207 1.0053157516739972


Теперь можно смело утверждать, что M = m (ведь в противном случае M / m >= 2, но m больше любого элемента из A (кроме, возможно, первого). То есть в таком случае частное M / max(arr[1:]) > M / m >= 2, что неверно).

In [6]:
m = M

Теперь, зная m, найдём a. d[1] = a * d[0] (mod m), то есть d[1] = a * d[0] + b * m, где b - некоторое целое число. Посмотрим на НОД d[0] и m. 

In [7]:
np.gcd(diff_arr[0], m)

1

Получили 1, значит можно выразить 1 как линейную комбинацию d[0] и m. Напишем функцию, которая по двум данным числам определяет их НОД, а заодно и находит коэффициенты его линейного разложения.

In [8]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        (g, x, y) = egcd(b % a, a)
        return (g, y - (b // a) * x, x)

Функция по паре (a, b) возвращает тройку чисел (g, x, y) такую, что g = НОД(a, b) и g = a * x + b * y.

У алгоритма две возможности. При a = 0 НОД(0, b) = b, а b = 0 * a + 1 * b. В противном случае можно поделить b на a с остатком, НОД g останется тем же. Имеем g = (b % a) * x + a * y = (b - a * (b // a)) * x + a * y = a * (y - (b // a) * x) + b * x, то есть возвращаем (g, y - (b // a) * x, x).

Конечность алгоритма обеспечивается следующими наблюдениями: 1) При первом рекурсивном вызове и далее первый аргумент меньше второго. 2) Это означает, что во всех вызовах, начиная со второго, a < b, следовательно b % a + a < a + b, то есть сумма аргументов с каждым рекурсивным вызовом уменьшается хотя бы на 1, а сами они остаются неотрицательными целыми. Значит, данная функция закончит работать при любых неотрицательных целых a и b.

Теперь найдём a (оно определено с точностью до остатка при делении на m).

In [9]:
diff_arr[0]

-18991396032

Применим функцию к модулю и возьмём со знаком минус (1 = u * d[0] + v * m = (-u) * (-d[0]) + v * m), затем умножим на d[1] и возьмём остаток при делении на m.

In [10]:
egcd(abs(diff_arr[0]), m)

(1, 24305623200, -3358565417)

In [11]:
a = (-egcd(abs(diff_arr[0]), m)[1] * diff_arr[1]) % m
a

31450092817

Теперь найдём c (оно тоже определено с точностью до остатка при делении на m): a[1] = a * a[0] + c (mod m), c = a[1] - a * a[0] (mod m).

In [12]:
c = (arr[1] - arr[0] * a) % m

Теперь проверим правильность нахождения параметров LCG.

In [13]:
test_arr = np.zeros(len(arr) + 1, dtype = object)

In [14]:
test_arr[0] = arr[0]
for i in range(1, len(arr) + 1):
    test_arr[i] = (test_arr[i - 1] * a + c) % m

In [15]:
np.max(np.abs(test_arr[:-1] - arr)) 

0

Отлично, массивы совпали. Ответ лежит в последнем элементе test_arr.

In [16]:
answer = test_arr[-1]
answer

2982350473